In [71]:
import random
import numpy as np

In [72]:
dataset=[10,11]
learningRate=0.0002
thresh=1

class Layer:
    
    def __init__(self,neuronCount=None,position=None,neuronvals=None):
        
        self.neuronCount = neuronCount
        self.position = position
        self.neuronvals = []
        self.weights = []
        self.bias = []
        self.NodeDeltas=[] 

        #set by setintialweights and setnodedelta
        self.previousLayer =None
        self.AfterLayer =None

        for i in range(self.neuronCount):

            # self.bias.append(random.uniform(0.0001,1))
            self.bias.append(1)

        if position==1:
            self.neuronvals = neuronvals
    
    def setInitialWeights(self,previousLayer):

        self.previousLayer = previousLayer
        test =[
            [1,2,3,4]
        ]
        for i in range(self.neuronCount):
            temp = []
            for j in range(previousLayer.neuronCount):
                temp.append(random.uniform(0.0001,1))
                # temp.append(test[0][i+j])

            self.weights.append(temp)

    def setNodeDelta(self,AfterLayer):
        """""
        set nodedelta value for layers
        """""
        self.AfterLayer = AfterLayer
        if self.position == -1:
            # outputlayer
            if len(self.NodeDeltas)==0:
                for i in range(self.neuronCount):

                    nodedelta = self.neuronvals[i]-dataset[i] #exepectd to change dimenston CHNGDIMN
                    self.NodeDeltas.append(nodedelta)
            else:
                for i in range(self.neuronCount):

                    nodedelta = self.neuronvals[i]-dataset[i] #exepectd to change dimenston CHNGDIMN
                    self.NodeDeltas[i]=nodedelta
        else:
            # hidden layers
            if len(self.NodeDeltas)==0:

                for i in range(self.neuronCount):
                    nodedelta= 0
                    for j in range(AfterLayer.neuronCount):
                        nodedelta = nodedelta+AfterLayer.NodeDeltas[j]*self.weights[i][j]
                    self.NodeDeltas.append(nodedelta)

            else:
                    for i in range(self.neuronCount):
                        nodedelta= 0
                        for j in range(AfterLayer.neuronCount):
                            nodedelta = nodedelta+AfterLayer.NodeDeltas[j]*self.weights[i][j]
                        self.NodeDeltas[i]=nodedelta

    
    def updateWeightsandBias(self):

        len_weight = len(self.weights[0])# based on previous layer of no(neurons) lenof weights is set

        for i in range(self.neuronCount):

            for j in range(len_weight):
                
                new_Weight = self.weights[i][j] - learningRate * self.NodeDeltas[i] * self.previousLayer.neuronvals[j]
                self.weights[i][j] = new_Weight
            
            new_Bias = self.bias[i] - learningRate * self.NodeDeltas[i]
            self.bias[i]=new_Bias        

In [73]:
class Ann:

    def __init__(self):
        self.LayerArr = []

    def strap(self,layer1=None,layer2=None):
        layer2.setInitialWeights(layer1)

    def forward(self,layer1=None,layer2=None):  
        if len(layer2.neuronvals)==0:
            for i in range(layer2.neuronCount):
                layer2.neuronvals.append(np.dot(layer1.neuronvals,layer2.weights[i])+layer2.bias[i])
        else:
            for i in range(layer2.neuronCount):
                layer2.neuronvals[i]=np.dot(layer1.neuronvals,layer2.weights[i]+layer2.bias[i])
            
    def backPropagate(self,currentLayer=None,AfterLayer=None):

        currentLayer.setNodeDelta(AfterLayer)
        currentLayer.updateWeightsandBias()
        
    def compile(self):

        for i in range(len(self.LayerArr)-1):

            self.strap(layer1=self.LayerArr[i],layer2=self.LayerArr[i+1])
    
    def findError(self):
        error = 0
        for i in range(len(self.LayerArr[-1].neuronvals)):#CHNGDIMN
            error = error+(dataset[i]-self.LayerArr[-1].neuronvals[i])**2
        error = 0.5*error
        return error
    
    def Train(self):

        errorrate = 9999999 
        layercount = len(self.LayerArr)

        while (errorrate > thresh):

            #forward propagation
            for i in range(layercount-1):
                self.forward(layer1=self.LayerArr[i],layer2=self.LayerArr[i+1])
            
            #error analysis
            errorrate = self.findError()
            # print(errorrate)
            if(errorrate <thresh):
                print("optimised")
                print(self.LayerArr[-1].neuronvals)
                break
          

            #backward propagation
            currentLayeridx = layercount-1  
            for i in range(layercount-1):

                if self.LayerArr[currentLayeridx].position ==-1:# handling output layer 
                    self.backPropagate(currentLayer=self.LayerArr[currentLayeridx])
                    print(self.LayerArr[currentLayeridx].neuronvals)
                    currentLayeridx=currentLayeridx-1
                else:
                    self.backPropagate(currentLayer=self.LayerArr[currentLayeridx],AfterLayer=self.LayerArr[currentLayeridx+1])
                    currentLayeridx=currentLayeridx-1
            
                    
    def bind(self,layer=None):
        self.LayerArr.append(layer)

In [74]:

# inputl = Layer(2,1,[2,3])
inputl = Layer(neuronCount=2,position=1,neuronvals=[2,3])
hidden = Layer(neuronCount=2,position=2,neuronvals=[])
# hidden2 = Layer(neuronCount=2,position=3,neuronvals=[]) 
output = Layer(neuronCount=2,position=-1,neuronvals=[])

Ann = Ann()
Ann.bind(layer=inputl)
Ann.bind(layer=hidden)
# Ann.bind(layer=hidden2)
Ann.bind(layer=output)
Ann.compile()
Ann.Train()

[2.65702140055576, 4.085144185670522]
[17.83979095568077, 21.208679791360726]
[17.604666910050028, 20.909453472055517]
[17.378802398003167, 20.621866747184658]
[17.16169621631781, 20.345292337328516]
[16.95288318466973, 20.079147998155985]
[16.75193091366081, 19.822892481131774]
[16.558436917846954, 19.576021925336086]
[16.372026031322747, 19.338066627364825]
[16.192348089305906, 19.10858814363265]
[16.01907584414541, 18.887176685624475]
[15.851903088405432, 18.673448773923425]
[15.690542961277092, 18.467045121341616]
[15.534726417643913, 18.26762871932025]
[15.384200841759284, 18.074883105054823]
[15.238728789754084, 17.888510789624856]
[15.098086847138386, 17.708231829838667]
[14.962064589140034, 17.533782528601442]
[14.830463633175722, 17.364914250430232]
[14.703096774009762, 17.201392340313333]
[14.579787193250667, 17.042995135479487]
[14.460367735789283, 16.88951306083428]
[14.344680246614942, 16.740747799861204]
[14.23257496217414, 16.596511533694965]
[14.123909951074856, 16.4566